In [1]:
import os
import sys
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_dir = os.path.dirname(notebook_dir)

if project_dir not in sys.path:
    sys.path.append(project_dir)

import pandas as pd
import numpy as np

In [2]:
all_fighter_cluster = pd.read_csv(r"https://raw.githubusercontent.com/john-adeojo/ufc_styles/main/data/02_intermediate/all_fighter_clusters.csv")
fighter_data = pd.read_csv(r"https://github.com/john-adeojo/ufc_styles/blob/main/data/02_intermediate/fighter_data.csv?raw=true")
# all_fighter_cluster.drop(columns='Unnamed: 0', inplace=True)

In [3]:
analysis_data_base = fighter_data.merge(right=all_fighter_cluster[['cluster','specific_cluster', 'Fighter_dims', 'weight_class']], how='left', left_on=['Fighter_dims', 'weight_class'], right_on=['Fighter_dims', 'weight_class'])
analysis_data_base = analysis_data_base.dropna(subset=['cluster'])
analysis_data_opp = analysis_data_base.copy()
analysis_data_opp['opp_specific_cluster'] = analysis_data_opp['specific_cluster']
analysis_data = analysis_data_base.merge(right=analysis_data_opp[['weight_class', 'Fighter_dims', 'date', 'opp_specific_cluster']], how='inner', left_on=['weight_class', 'Opponent', 'date'], right_on=['weight_class', 'Fighter_dims', 'date'] )
analysis_data_opposing_styles = analysis_data.loc[analysis_data['opp_specific_cluster'] != analysis_data['specific_cluster']].copy()
analysis_data_opposing_styles = analysis_data_opposing_styles.loc[analysis_data_opposing_styles['cluster'] != -1]

In [4]:
analysis_data_opposing_styles = analysis_data_opposing_styles.loc[analysis_data_opposing_styles['corner'] == 'Red'].copy()
analysis_data_opposing_styles

,R_fighter,B_fighter,date,Winner,weight_class,avg_KD,avg_opp_KD,avg_SUB_ATT,avg_opp_SUB_ATT,avg_REV,...,Stance_Southpaw,Stance_Switch,Stance_nan,fighter,method,fighter_result,cluster,specific_cluster,Fighter_dims_y,opp_specific_cluster
4306,Arman Tsarukyan,Mateusz Gamrot,2022-06-25,Blue,Lightweight,0.000537,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,Arman Tsarukyan,U-DEC,Loss by U-DEC,1.0,1_Lightweight,Mateusz Gamrot,0_Lightweight
4318,Joaquin Buckley,Albert Duraev,2022-06-18,Red,Middleweight,0.000576,0.000266,0.000000,0.000000,0.000000,...,1.0,0.0,0.0,Joaquin Buckley,KO/TKO,Win by KO/TKO,0.0,0_Middleweight,Albert Duraev,1_Middleweight
4325,Roman Dolidze,Kyle Daukaus,2022-06-18,Red,Middleweight,0.000153,0.000000,0.001068,0.000305,0.000305,...,0.0,0.0,0.0,Roman Dolidze,KO/TKO,Win by KO/TKO,1.0,1_Middleweight,Kyle Daukaus,0_Middleweight
4327,Jack Della Maddalena,Ramazan Emeev,2022-06-11,Red,Welterweight,0.005587,0.000000,0.000000,0.000000,0.000000,...,0.0,1.0,0.0,Jack Della Maddalena,KO/TKO,Win by KO/TKO,1.0,1_Welterweight,Ramazan Emeev,0_Welterweight
4340,Niklas Stolze,Benoit Saint Denis,2022-06-04,Blue,Lightweight,0.000000,0.001033,0.000000,0.002066,0.000000,...,0.0,1.0,0.0,Niklas Stolze,SUB,Loss by SUB,1.0,1_Lightweight,Benoit Saint Denis,0_Lightweight
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8589,Chuck Liddell,Vitor Belfort,2002-06-22,Red,LightHeavyweight,0.000487,0.000000,0.000000,0.000061,0.000020,...,0.0,0.0,0.0,Chuck Liddell,U-DEC,Win by U-DEC,1.0,1_LightHeavyweight,Vitor Belfort,0_LightHeavyweight
8593,Jens Pulver,BJ Penn,2002-01-11,Red,Lightweight,0.000103,0.000000,0.000488,0.001233,0.000051,...,1.0,0.0,0.0,Jens Pulver,M-DEC,Win by M-DEC,0.0,0_Lightweight,BJ Penn,1_Lightweight
8594,BJ Penn,Caol Uno,2001-11-02,Red,Lightweight,0.002179,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,BJ Penn,KO/TKO,Win by KO/TKO,1.0,1_Lightweight,Caol Uno,0_Lightweight
8595,Matt Lindland,Phil Baroni,2001-11-02,Red,Middleweight,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.0,0.0,Matt Lindland,M-DEC,Win by M-DEC,0.0,0_Middleweight,Phil Baroni,1_Middleweight


In [5]:
import pandas as pd

# Assuming your DataFrame is named 'analysis_data_opposing_styles'
# and the columns to combine are 'specific_cluster' and 'opp_specific_cluster'

def create_style_matchup(row):
    # Combine the two columns and sort them alphabetically
    matchup = sorted([row['specific_cluster'], row['opp_specific_cluster']])
    
    # Join them with ' vs. ' and return the result
    return matchup[0] + ' vs. ' + matchup[1]

# Apply the custom function to each row and create the new 'style_matchup' column
analysis_data_opposing_styles['style_matchup'] = analysis_data_opposing_styles.apply(create_style_matchup, axis=1)

# Display the DataFrame
analysis_data_opposing_styles.sort_values(by='weight_class', inplace=True)


In [6]:
weight_class = list(analysis_data_opposing_styles['weight_class'].drop_duplicates())
weight_class

['Bantamweight',
 'Featherweight',
 'LightHeavyweight',
 'Lightweight',
 'Middleweight',
 'Welterweight']

In [7]:
analysis_data_opposing_styles.loc[analysis_data_opposing_styles['style_matchup'] == '0_Lightweight vs. 1_Lightweight']

,R_fighter,B_fighter,date,Winner,weight_class,avg_KD,avg_opp_KD,avg_SUB_ATT,avg_opp_SUB_ATT,avg_REV,...,Stance_Switch,Stance_nan,fighter,method,fighter_result,cluster,specific_cluster,Fighter_dims_y,opp_specific_cluster,style_matchup
8412,Nate Diaz,Junior Assuncao,2007-09-19,Red,Lightweight,0.000000,0.000000,0.003125,0.003125,0.000000,...,0.0,0.0,Nate Diaz,SUB,Win by SUB,0.0,0_Lightweight,Junior Assuncao,1_Lightweight,0_Lightweight vs. 1_Lightweight
6893,Michael Chiesa,Mitch Clarke,2015-04-04,Red,Lightweight,0.000114,0.000000,0.000626,0.000911,0.000228,...,0.0,0.0,Michael Chiesa,U-DEC,Win by U-DEC,0.0,0_Lightweight,Mitch Clarke,1_Lightweight,0_Lightweight vs. 1_Lightweight
8418,Dennis Siver,Naoyuki Kotani,2007-09-08,Red,Lightweight,0.000000,0.000000,0.000000,0.012346,0.000000,...,0.0,0.0,Dennis Siver,KO/TKO,Win by KO/TKO,1.0,1_Lightweight,Naoyuki Kotani,0_Lightweight,0_Lightweight vs. 1_Lightweight
6871,Chad Laprise,Bryan Barberena,2015-04-25,Red,Lightweight,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,Chad Laprise,U-DEC,Win by U-DEC,1.0,1_Lightweight,Bryan Barberena,0_Lightweight,0_Lightweight vs. 1_Lightweight
7786,Clay Guida,Benson Henderson,2011-11-12,Blue,Lightweight,0.000003,0.000062,0.000759,0.001021,0.000664,...,0.0,0.0,Clay Guida,U-DEC,Loss by U-DEC,1.0,1_Lightweight,Benson Henderson,0_Lightweight,0_Lightweight vs. 1_Lightweight
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4499,Diego Ferreira,Mateusz Gamrot,2021-12-18,Blue,Lightweight,0.000012,0.000006,0.000260,0.000000,0.001879,...,0.0,0.0,Diego Ferreira,KO/TKO,Loss by KO/TKO,1.0,1_Lightweight,Mateusz Gamrot,0_Lightweight,0_Lightweight vs. 1_Lightweight
6380,Francisco Trinaldo,Paul Felder,2016-09-24,Red,Lightweight,0.000158,0.000620,0.000012,0.000022,0.000019,...,0.0,0.0,Francisco Trinaldo,KO/TKO,Win by KO/TKO,0.0,0_Lightweight,Paul Felder,1_Lightweight,0_Lightweight vs. 1_Lightweight
6221,Edson Barboza,Beneil Dariush,2017-03-11,Red,Lightweight,0.000014,0.000005,0.000002,0.000161,0.000000,...,0.0,0.0,Edson Barboza,KO/TKO,Win by KO/TKO,1.0,1_Lightweight,Beneil Dariush,0_Lightweight,0_Lightweight vs. 1_Lightweight
4346,Omar Morales,Uros Medic,2022-05-21,Blue,Lightweight,0.000089,0.000177,0.000000,0.000709,0.001417,...,0.0,0.0,Omar Morales,KO/TKO,Loss by KO/TKO,1.0,1_Lightweight,Uros Medic,0_Lightweight,0_Lightweight vs. 1_Lightweight


In [8]:
from scripts.chisquareanalysis import ChiSquareAnalysis

# Assuming weight_class is a list of weight classes

# Initialize an empty dictionary to store the results
results_dict = {}

for weights in weight_class:
    analysis = ChiSquareAnalysis(df=analysis_data_opposing_styles, category=weights)
    result = analysis.run_chisquare_analysis('fighter_result')
    
    # Save the result in the dictionary with the weight as the key
    # print(weights)
    # print(result)

# Display the results dictionary

Bantamweight
Chi2 Stat: 0.0
P Value: 1.0
Degrees of Freedom: 0
fighter_result                     Draw by DRAW  Loss by KO/TKO  \
style_matchup                                                     
0_Bantamweight vs. 1_Bantamweight             1              15   

fighter_result                     Loss by S-DEC  Loss by SUB  Loss by U-DEC  \
style_matchup                                                                  
0_Bantamweight vs. 1_Bantamweight              4            7              7   

fighter_result                     Win by KO/TKO  Win by S-DEC  Win by SUB  \
style_matchup                                                                
0_Bantamweight vs. 1_Bantamweight             14             5          13   

fighter_result                     Win by U-DEC  
style_matchup                                    
0_Bantamweight vs. 1_Bantamweight            24  
fighter_result                      style_matchup  Draw by DRAW  \
0               0_Bantamweight vs. 1_Banta

In [9]:
fighter_result = list(analysis_data_opposing_styles['fighter_result'].drop_duplicates())

In [10]:
from scripts.chisquareanalysis import ChiSquareAnalysis
analysis = ChiSquareAnalysis(df=analysis_data_opposing_styles, category='Bantamweight')
result_batamweight = analysis.run_chisquare_analysis('fighter_result')
result_batamweight

Bantamweight
Chi2 Stat: 0.0
P Value: 1.0
Degrees of Freedom: 0
fighter_result                     Draw by DRAW  Loss by KO/TKO  \
style_matchup                                                     
0_Bantamweight vs. 1_Bantamweight             1              15   

fighter_result                     Loss by S-DEC  Loss by SUB  Loss by U-DEC  \
style_matchup                                                                  
0_Bantamweight vs. 1_Bantamweight              4            7              7   

fighter_result                     Win by KO/TKO  Win by S-DEC  Win by SUB  \
style_matchup                                                                
0_Bantamweight vs. 1_Bantamweight             14             5          13   

fighter_result                     Win by U-DEC  
style_matchup                                    
0_Bantamweight vs. 1_Bantamweight            24  
fighter_result                      style_matchup  Draw by DRAW  \
0               0_Bantamweight vs. 1_Banta

fighter_result,style_matchup,Draw by DRAW,Loss by KO/TKO,Loss by S-DEC,Loss by SUB,Loss by U-DEC,Win by KO/TKO,Win by S-DEC,Win by SUB,Win by U-DEC
0,0_Bantamweight vs. 1_Bantamweight,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
import seaborn as sns
import matplotlib as plt 

esult_batamweight_col = result_batamweight.drop(columns=['style_matchup']).columns

# Prepare the data for the heatmap
heatmap_data = result_batamweight.set_index('style_matchup')[result_batamweight_col]

# Create the heatmap
plt.figure(figsize=(8, 4))
sns.heatmap(heatmap_data, cmap='coolwarm', annot=True, linewidths=0.5, center=0)
plt.title('Fight Result by Style: Batamweight')
plt.xlabel('Result')
plt.ylabel('Style Matchup')
#plt.savefig(r"C:\Users\johna\anaconda3\envs\twitter-influence-env\twitter-influence\data\03_reports\sentiment_heatmap.png")
plt.show()

NameError: name 'result_batamweight_col' is not defined

In [ ]:
import requests
import json
# import style dims
url = 'https://github.com/john-adeojo/ufc_styles/raw/main/data/02_intermediate/style_dims_names.json'

# Fetch the JSON file content
response = requests.get(url)

# Check if the request was successful (HTTP status code 200)
if response.status_code == 200:
    try:
        # Load the content into a list
        style_dims_names = json.loads(response.text)
        print("Data loaded successfully:")
    except json.JSONDecodeError as e:
        print(f"Failed to decode JSON: {e}")
else:
    print(f"Failed to fetch JSON file: {response.status_code}")

In [ ]:
style_dims_names_2 = style_dims_names
style_dims_names_2.append('specific_cluster')

In [ ]:
selected_clusters = ['0_Bantamweight', '1_Bantamweight', '2_Bantamweight']
filtered_data = all_fighter_cluster[all_fighter_cluster['specific_cluster'].isin(selected_clusters)]
analytics_for_radar = filtered_data[style_dims_names_2].groupby(['specific_cluster']).mean()
analytics_for_radar.reset_index(inplace=True)
analytics_for_radar

In [ ]:
analytics_for_radar.info()

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Replace this with your actual dataframe
# analytics_for_radar = ...

# Select dimensions to plot
selected_dimensions = analytics_for_radar.drop(columns='specific_cluster').columns

def plot_fighters_radar_chart(data=analytics_for_radar, dimensions=selected_dimensions):
    fig = go.Figure()

    for specific_cluster in data['specific_cluster'].unique():
        cluster_data = data[data['specific_cluster'] == specific_cluster]

        fig.add_trace(go.Scatterpolar(
            r=cluster_data[dimensions].values.tolist()[0],
            theta=dimensions,
            fill='toself',
            name=specific_cluster,
            opacity=0.5,
            line=dict(width=0.8)
        ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, data[dimensions].values.astype(float).max()]  # Set the range based on the maximum value in the data
            ),
            angularaxis=dict(
                rotation=90,
                direction="counterclockwise"
            ),
        ),
        showlegend=True,
        width=1400,
        height=1000
    )

    fig.show()

plot_fighters_radar_chart()
